In [1]:
import threading
import time

In [24]:
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")
    
if __name__ == '__main__':
    thread_1 = threading.Thread(target=T1_job,name='T1')
    thread_2 = threading.Thread(target=T2_job,name='T2')
    thread_1.start()
    thread_2.start()
    thread_1.join()
    thread_2.join()
    print('all done\n')

T1 start

T2 start

T2 finish

T1 finish

all done



In [10]:
from queue import Queue

def job(l,q):
    for i in range (len(l)):
        l[i] = l[i]**2
        time.sleep(0.1)
    q.put(l)

def multithreading(): #在线程中是不能返回一个值得
    q =Queue()
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    for i in range(4):
        t = threading.Thread(target=job,args=(data[i],q))
        t.start()
        threads.append(t)
    for thread in threads: #不加的话线程会乱
        thread.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)

if __name__ =='__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [25, 25, 25], [16, 16, 16]]


In [13]:
## Global Interpreter Lock(GIL)
# python中尽管视觉上面是可以实现多线程的，但是其实只有每个时间都只会有一个线程执行
# GIL知识影响到严重依赖GPU的程序，但是如果程序大部分依赖I/O
# 因为多线程在一个线程在进行I/O读写的时候会执行另外一个线程的GPU执行，
# 所以如果做网络交互，大部分时间都设计I/O，所以很适合多线程运算

import threading
from queue import Queue
import copy
import time

def job(l,q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target = job,args=(copy.copy(l),q),name='T%i'%i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)
    
def normal(l):
    total = sum(l)
    print(total)
    
if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal:',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading:',time.time()-s_t)

1999998000000
normal: 0.155411958694458
1999998000000
multithreading: 0.13866758346557617


In [19]:
# 线程锁
# lock
def job1():
    global A,lock
    lock.acquire()
    for i in range(10):
        A += 1
        print('job1:',A)
    lock.release()
        
def job2():
    global A,lock
    lock.acquire()
    for i in range(10):
        A += 10
        print('job2:', A)
    lock.release()

if __name__ == '__main__':
    lock = threading.Lock()
    A = 0
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1: 1
job1: 2
job1: 3
job1: 4
job1: 5
job1: 6
job1: 7
job1: 8
job1: 9
job1: 10
job2: 20
job2: 30
job2: 40
job2: 50
job2: 60
job2: 70
job2: 80
job2: 90
job2: 100
job2: 110


In [20]:
# 创建多进程：
# 可以利用CPU的多核处理
import multiprocessing as mp
import threading as td


In [21]:
# 添加进程process
def job(a,d):
    print('aaaa')

if __name__ == '__main__':
    #t1 = td.Thread(target=job,args(1,2))
    p1 = mp.Process(target=job,args=(1,2))
    #t1.start()
    p1.start()
    #t1.join()
    p1.join()

In [ ]:
# 利用Queue()
def job(q):
    res = 0
    for i in range(1000):
        res += i+i**2+i**3
    q.put(res) #queue

if __name__ == '__main__':
    q = mp.Queue()
    p1 = mp.Process(target=job,args=(q,))
    p2 = np.Process(target=job,args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print(res1+res2)

In [ ]:
import multiprocessing as mp

def job(q):
    res = 0
    for i in range(1000000):
        res += i + i**2 + i**3
    q.put(res) # queue
    
def multicore():
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print('multicore:',res1 + res2)

import threading as td

def multithread():
    q = mp.Queue() # thread可放入process同样的queue中
    t1 = td.Thread(target=job, args=(q,)) # 因为q是可跌代的，所以要加','
    t2 = td.Thread(target=job, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    res1 = q.get()
    res2 = q.get()
    print('multithread:', res1 + res2)
    
def normal():
    res = 0
    for _ in range(2):
        for i in range(1000000):
            res += i + i**2 + i**3
    print('normal:', res)
    
import time

if __name__ == '__main__':
    st = time.time()
    normal()
    st1 = time.time()
    print('normal time:', st1 - st)
    multithread()
    st2 = time.time()
    print('multithread time:', st2 - st1)
    multicore()
    print('multicore time:', time.time() - st2)
    

"""
# range(10000000)
('normal:', 4999999666666716666660000000L)
('normal time:', 40.041773080825806)
('thread:', 4999999666666716666660000000L)
('multithread time:', 41.777158975601196)
('multicore:', 4999999666666716666660000000L)
('multicore time:', 22.4337899684906)
"""


In [ ]:
# 进程池:把所有要执行的东西放到一个池子里，python自行解决多进程的问题

import multiprocessing as mp

def job(x):
    return x*x

pool = mp.Pool(process = 3) # 在pool中的函数有return，与process 中不同
                            # 自定义CPU核的个数为3

res = pool.map(job,range(10)) # map() 中需要放入函数和需要迭代运算的值，它会自动分配给CPU的核

def multicore():
    pool = mp.Pool()
    res = pool.map(job,range(10))
    print(res)
    
# apply_async() 也可以返回结果，但是这个函数只能传递一个值，它只会放入一个核进行运算
# 注意传入值后面由于是可迭代的，所以传入一个值的后面加，
def multicore1():
    pool = mp.Pool() 
    res = pool.map(job, range(10))
    print(res)
    res = pool.apply_async(job, (2,)) # 只能传入一个参数，如果传入（2，3，4，）会报错
    # 用get获得结果
    print(res.get())
    # 迭代器，i=0时apply一次，i=1时apply一次等等
    multi_res = [pool.apply_async(job, (i,)) for i in range(10)]
    # 从迭代器中取出
    print([res.get() for res in multi_res])

if __name__ == '__main__':
    multicore()
    
    

In [ ]:
# 共享内存(shared memory)
# https://docs.python.org/3.5/library/array.html
# 也可以用进程中通信实现进程中的变量的交互
import multiprocessing as mp
# 两类，value和array类
value1 = mp.Value('i',0)
value2 = mp.Value('d',3.14) 
array = mp.Array('i',[1,2,3,4]) # 只能是一维列表

| Type code | C Type             | Python Type       | Minimum size in bytes |
| --------- | ------------------ | ----------------- | --------------------- |
| `'b'`     | signed char        | int               | 1                     |
| `'B'`     | unsigned char      | int               | 1                     |
| `'u'`     | Py_UNICODE         | Unicode character | 2                     |
| `'h'`     | signed short       | int               | 2                     |
| `'H'`     | unsigned short     | int               | 2                     |
| `'i'`     | signed int         | int               | 2                     |
| `'I'`     | unsigned int       | int               | 2                     |
| `'l'`     | signed long        | int               | 4                     |
| `'L'`     | unsigned long      | int               | 4                     |
| `'q'`     | signed long long   | int               | 8                     |
| `'Q'`     | unsigned long long | int               | 8                     |
| `'f'`     | float              | float             | 4                     |
| `'d'`     | double             | float             | 8                     |

In [ ]:
# 进程锁
# 不加进程锁
import multiprocessing as mp
import time

def job(v, num):
    for _ in range(5):
        time.sleep(0.1) # 暂停0.1秒，让输出效果更明显
        v.value += num # v.value获取共享变量值
        print(v.value, end="")
        
def multicore():
    v = mp.Value('i', 0) # 定义共享变量
    p1 = mp.Process(target=job, args=(v,1))
    p2 = mp.Process(target=job, args=(v,3)) # 设定不同的number看如何抢夺内存
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    
if __name__ == '__main__':
    multicore()

In [ ]:
# 加进程锁
def job(v, num, l):
    l.acquire() # 锁住
    for _ in range(5):
        time.sleep(0.1) 
        v.value += num # 获取共享内存
        print(v.value)
    l.release() # 释放

def multicore():
    l = mp.Lock() # 定义一个进程锁
    v = mp.Value('i', 0) # 定义共享内存
    p1 = mp.Process(target=job, args=(v,1,l)) # 需要将lock传入
    p2 = mp.Process(target=job, args=(v,3,l)) 
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__ == '__main__':
    multicore()